# 直流电源

在这一节中，我们将考虑如何在Modelica实现直流电源模型。我们将再一次展示，怎么重构无层级的系统模型，使得可重用的子系统模型能得到利用。

## 无层级电源模型

在这里，我们的无层级系统模型是如下的直流电源电路：

<img src="./figures8/FlatCircuit.svg" width=800 align=center>


模型的Modelica实现则如下：

```Modelica
within ModelicaByExample.Subsystems.PowerSupply.Examples;
model FlatCircuit "A model with power source, AC-DC conversion and load in one diagram"
  import Modelica.Electrical.Analog;
  Analog.Sources.SineVoltage wall_voltage(V=120, freqHz=60)
    annotation (Placement(transformation(
        extent={{-10,-10},{10,10}},
        rotation=270, origin={-90,0})));
  Analog.Ideal.IdealClosingSwitch switch(Goff=0)
    annotation (Placement(transformation(extent={{-80,30},{-60,50}})));
  Analog.Ideal.IdealTransformer transformer(Lm1=1, n=10, considerMagnetization=false)
    annotation (Placement(transformation(extent={{-50,0},{-30,20}})));
  Analog.Ideal.IdealDiode D1(Vkne0e=0, Ron=1e-5, Goff=1e-5)
    annotation (Placement(
        transformation(
        extent={{-10,-10},{10,10}},
        rotation=45, origin={-12,20})));
  Analog.Basic.Capacitor capacitor(C=1e-2)
    annotation (Placement(
        transformation(
        extent={{-10,-10},{10,10}},
        rotation=270, origin={60,-10})));
  Analog.Basic.Resistor load(R=100)
    annotation (Placement(
        transformation(
        extent={{-10,-10},{10,10}},
        rotation=270, origin={100,-10})));
  Modelica.Blocks.Sources.BooleanStep step(startTime=0.25)
    annotation (Placement(transformation(extent={{-100,50},{-80,70}})));
  Analog.Ideal.IdealDiode D2(Vknee=0, Ron=1e-5, Goff=1e-5)
    annotation (Placement(
        transformation(
        extent={{-10,-10},{10,10}},
        rotation=-45, origin={12,20})));
  Analog.Ideal.IdealDiode D3(Vknee=0, Ron=1e-5, Goff=1e-5)
    annotation (Placement(
        transformation(
        extent={{-10,-10},{10,10}},
        rotation=-45, origin={-12,0})));
  Analog.Ideal.IdealDiode D4(Vknee=0, Ron=1e-5, Goff=1e-5)
    annotation (Placement(
        transformation(
        extent={{-10,-10},{10,10}},
        rotation=45, origin={12,0})));
  Analog.Basic.Ground ground1
    annotation (Placement(transformation(extent={{-100,-52},{-80,-32}})));
equation
  connect(switch.p, wall_voltage.p) annotation (Line(
      points={{-80,40},{-90,40},{-90,10}},
      color={0,0,255}, smooth=Smooth.None));
  connect(switch.n, transformer.p1) annotation (Line(
      points={{-60,40},{-50,40},{-50,15}},
      color={0,0,255}, smooth=Smooth.None));
  connect(step.y, switch.control) annotation (Line(
      points={{-79,60},{-70,60},{-70,47}},
      color={255,0,255}, smooth=Smooth.None));
  connect(D3.n, D4.p) annotation (Line(
      points={{-4.92893,-7.07107},{-2.46446,-7.07107},{-2.46446,-7.07107},{
          1.09406e-006,-7.07107},{1.09406e-006,-7.07107},{4.92893,-7.07107}},
      color={0,0,255}, smooth=Smooth.None));
  connect(D1.n, D2.p) annotation (Line(
      points={{-4.92893,27.0711},{2.18813e-006,28},{4.92893,28},{4.92893, 27.0711}},
      color={0,0,255}, smooth=Smooth.None));
  connect(D1.p, D3.p) annotation (Line(
      points={{-19.0711,12.9289},{-20,10},{-19.0711,7.07107}},
      color={0,0,255}, smooth=Smooth.None));
  connect(D2.n, D4.n) annotation (Line(
      points={{19.0711,12.9289},{19.0711,11.4644},{19.0711,11.4644},{19.0711,10},
          {19.0711,7.07107},{19.0711,7.07107}},
      color={0,0,255}, smooth=Smooth.None));
  connect(transformer.p2, D1.n) annotation (Line(
      points={{-30,15},{-30,34},{0,34},{0,27.0711},{-4.92893,27.0711}},
      color={0,0,255}, smooth=Smooth.None));
  connect(D4.p, transformer.n2) annotation (Line(
      points={{4.92893,-7.07107},{0,-7.07107},{0,-20},{-30,-20},{-30,5}},
      color={0,0,255}, smooth=Smooth.None));
  connect(wall_voltage.n, transformer.n1) annotation (Line(
      points={{-90,-10},{-90,-20},{-50,-20},{-50,5}},
      color={0,0,255}, smooth=Smooth.None));
  connect(wall_voltage.n, ground1.p) annotation (Line(
      points={{-90,-10},{-90,-32}},
      color={0,0,255}, smooth=Smooth.None));
  connect(transformer.n2, ground1.p) annotation (Line(
      points={{-30,5},{-30,-32},{-90,-32}},
      color={0,0,255}, smooth=Smooth.None));
  connect(load.p, D2.n) annotation (Line(
      points={{100,0},{100,12},{20,12},{20,12.9289},{19.0711,12.9289}},
      color={0,0,255}, smooth=Smooth.None));
  connect(load.p, capacitor.p) annotation (Line(
      points={{100,0},{100,12},{60,12},{60,0}},
      color={0,0,255}, smooth=Smooth.None));
  connect(D1.p, capacitor.n) annotation (Line(
      points={{-19.0711,12.9289},{-24,12.9289},{-24,-32},{60,-32},{60,-20}},
      color={0,0,255}, smooth=Smooth.None));
  connect(load.n, capacitor.n) annotation (Line(
      points={{100,-20},{100,-32},{60,-32},{60,-20}},
      color={0,0,255}, smooth=Smooth.None));
end FlatCircuit;
```
电源把交流输入电压（120V，60Hz）经过整流后传入低通滤波器。对模型进行仿真后，我们可以看到load电阻上的电压如下：


<img src="./figures8/FC.png" width=1600 align=center>

请注意，我们这里的目标是12伏的输出电压。但是，输出信号的质量会随着电源负载的增大而变差。在这个模拟里，负载最初为零（由于连接到电源的负载开关为开路） 。但是，当开关闭合，电流开始流过负载（名为load的电阻）时，我们会开始看到一些错误。

## 有层级电源

首先以无层级版本为基础将一部分组件收集、整理成子系统模型，以简化系统模型。我们的系统级电路就变成了：

<img src="./figures8/SubsystemCircuit.svg" width=600 align=center>

上述系统模型采用了BasicPowerSupply模型。此子模型的简图如下所示：

<img src="./figures8/BasicPowerSupply.svg" width=600 align=center>

此可重用电源子系统模型的Modelica源代码为：

```Modelica
within ModelicaByExample.Subsystems.PowerSupply.Components;
model BasicPowerSupply "Power supply with transformer and rectifier"
  import Modelica.Electrical.Analog;
  parameter Modelica.SIunits.Capacitance C
    "Filter capacitance"
    annotation(Dialog(group="General"));
  parameter Modelica.SIunits.Conductance Goff=1.E-5
    "Backward state-off conductance (opened diode conductance)"
    annotation(Dialog(group="General"));
  parameter Modelica.SIunits.Resistance Ron=1.E-5
    "Forward state-on differential resistance (closed diode resistance)"
    annotation(Dialog(group="General"));
  parameter Real n
    "Turns ratio primary:secondary voltage"
    annotation(Dialog(group="Transformer"));
  parameter Boolean considerMagnetization=false
    "Choice of considering magnetization"
    annotation(Dialog(group="Transformer"));
  parameter Modelica.SIunits.Inductance Lm1
    "Magnetization inductance w.r.t. primary side"
    annotation(Dialog(group="Transformer", enable=considerMagnetization));

  Analog.Interfaces.NegativePin gnd
    "Pin to ground power supply"
    annotation (Placement(transformation(extent={{-110,-70},{-90,-50}})));
  Analog.Interfaces.PositivePin p
    "Positive pin on supply side"
    annotation (Placement(transformation(extent={{-110,50},{-90,70}})));
  Analog.Interfaces.PositivePin p_load
    "Positive pin for load side"
    annotation (Placement(transformation(extent={{90,50},{110,70}})));
  Analog.Interfaces.NegativePin n_load
    "Negative pin for load side"
    annotation (Placement(transformation(extent={{90,-70},{110,-50}})));
protected
  Analog.Ideal.IdealTransformer transformer(
    final n=n, final considerMagnetization=considerMagnetization,
    final Lm1=Lm1)
    annotation (Placement(transformation(extent={{-60,-10},{-40,10}})));
  Analog.Ideal.IdealDiode D1(final Vknee=0, final Ron=Ron, final Goff=Goff)
    annotation (Placement(
        transformation(
        extent={{-10,-10},{10,10}},
        rotation=45,
        origin={-2,10})));
  Analog.Basic.Capacitor capacitor(C=C)
    annotation (Placement(
        transformation(
        extent={{-10,-10},{10,10}},
        rotation=270,
        origin={70,-20})));
  Analog.Ideal.IdealDiode D2(final Vknee=0, final Ron=Ron, final Goff=Goff)
    annotation (Placement(
        transformation(
        extent={{-10,-10},{10,10}},
        rotation=-45,
        origin={22,10})));
  Analog.Ideal.IdealDiode D3(final Vknee=0, final Ron=Ron, final Goff=Goff)
    annotation (Placement(
        transformation(
        extent={{-10,-10},{10,10}},
        rotation=-45,
        origin={-2,-10})));
  Analog.Ideal.IdealDiode D4(final Vknee=0, final Ron=Ron, final Goff=Goff)
    annotation (Placement(
        transformation(
        extent={{-10,-10},{10,10}},
        rotation=45,
        origin={22,-10})));
equation
  connect(D3.n,D4. p) annotation (Line(
      points={{5.07107,-17.0711},{10,-18},{14.9289,-17.0711}},
      color={0,0,255},
      smooth=Smooth.None));
  connect(D1.n,D2. p) annotation (Line(
      points={{5.07107,17.0711},{10,18},{14.9289,18},{14.9289,17.0711}},
      color={0,0,255},
      smooth=Smooth.None));
  connect(D1.p,D3. p) annotation (Line(
      points={{-9.07107,2.92893},{-10,0},{-9.07107,-2.92893}},
      color={0,0,255},
      smooth=Smooth.None));
  connect(D2.n,D4. n) annotation (Line(
      points={{29.0711,2.92893},{30,0},{29.0711,-2.92893}},
      color={0,0,255},
      smooth=Smooth.None));
  connect(transformer.p2,D1. n) annotation (Line(
      points={{-40,5},{-40,24},{10,24},{10,17.0711},{5.07107,17.0711}},
      color={0,0,255},
      smooth=Smooth.None));
  connect(D4.p,transformer. n2) annotation (Line(
      points={{14.9289,-17.0711},{10,-17.0711},{10,-30},{-40,-30},{-40,-5}},
      color={0,0,255},
      smooth=Smooth.None));
  connect(D1.p, capacitor.n) annotation (Line(
      points={{-9.07107,2.92893},{-14,2.92893},{-14,-42},{70,-42},{70,-30}},
      color={0,0,255},
      smooth=Smooth.None));
  connect(transformer.n1, gnd) annotation (Line(
      points={{-60,-5},{-60,-60},{-100,-60}},
      color={0,0,255},
      smooth=Smooth.None));
  connect(transformer.n2, gnd) annotation (Line(
      points={{-40,-5},{-40,-60},{-100,-60}},
      color={0,0,255},
      smooth=Smooth.None));
  connect(transformer.p1, p) annotation (Line(
      points={{-60,5},{-60,60},{-100,60}},
      color={0,0,255},
      smooth=Smooth.None));
  connect(capacitor.p, D2.n) annotation (Line(
      points={{70,-10},{70,2.92893},{29.0711,2.92893}},
      color={0,0,255},
      smooth=Smooth.None));
  connect(capacitor.p, p_load) annotation (Line(
      points={{70,-10},{70,60},{100,60}},
      color={0,0,255},
      smooth=Smooth.None));
  connect(capacitor.n, n_load) annotation (Line(
      points={{70,-30},{70,-60},{100,-60}},
      color={0,0,255},
      smooth=Smooth.None));
end BasicPowerSupply;

```
模型里有几个有趣的地方需要注意。
- 首先，我们看到与先前相同的组织结构。参数和接口均为public，而内部组件则为protected。
- 我们也看到用Dialog标注来将参数组织为不同的组（这里是"General"和"Transformer"）。
- 我们还看到enable标注上使用了considerMagnetization参数。只有considerMagnetization为真的情况下，Lm1参数才会选择性地有效。

使用有层级系统模型，我们不出所料地得到与无层级版本相同的结果：

<img src="./figures8/SSC.png" width=600 align=center>


可以方便的扩展系统模型，包括额外的负载：

<img src="./figures8/AdditionalLoad.svg" width=800 align=center>

在这种情况下，如果我们仿真模型就可以看到额外负载将会对电源输出质量的影响：

<img src="./figures8/SSC_AL.png" width=800 align=center>

通过增大电源内的电容就可以减少的电压波动的幅度，例如：

<img src="./figures8/SSC_ALC.png" width=800 align=center>

但是，如果我们增加电容容量过大，我们会发现电源输出对负载变化的响应变得非常慢，如：

<img src="./figures8/SSC_ALC2.png" width=800 align=center>


## 结论

这个例子再次说明了组件的集合如何能组织成可重复使用的子系统模型。这个例子遵从了放置参数的最佳实践。而连接器在所得子系统模型的public区域。同时，内部组件则留在protected区域内。